In [ ]:
# Plots for section 5.3 (hydrogen mv)

## Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pypsa
import math
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
from shapely.geometry import Point, LineString
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
from datetime import date, datetime, time, timedelta
import matplotlib.ticker as mtick

# imported own functions
from utils import generation_links_bus, congestion_rent_link, nodal_balance

# imported own definitions
from utils import carrier_colors, carrier_renaming, carrier_renaming_reverse, c_tags
from utils import resistive_heater, gas_boiler, heat_pump,water_tanks_charger, water_tanks_discharger, solar_thermal
from utils import c_el_gen_s, c_el_con_s, c_h2_gen, c_h2_con
from utils import convert_ISO_3166_2_to_1

# general variables
font1 = {'fontname': 'Calibri'}
PLOT_DIR = 'C:/Users/Julian/Studies/Master/01 TU Berlin/3. Semester - Masterarbeit/MA Marktwerte FEE/data/plots/01_general/5.3_hydrogen_mv'
onshore_regions = gpd.read_file("../data/external/regions_onshore_elec_s_181.geojson")
offshore_regions = gpd.read_file("../data/external/regions_offshore_elec_s_181.geojson")
onshore_regions = onshore_regions.set_index('name')
offshore_regions = offshore_regions.set_index('name')

# Notebook Definitions
c1_groups = [resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal]
c1_groups_name = ["resistive heater", "gas boiler", "heat pump", "water tanks charger", "water tanks discharger",
                  "solar thermal"]
markers = ["v", "^", "<", ">", "1", "2", "3", "4", "*", "+", "d", "o", "|", "s", "P", "p", "h"]
# figsizes
s_w = 15
s_h = 6
s_shrink = 0.9
t_f_w = 15
t_f_h = 6

# new indices: cut off Fuel cell and SMR completely and SMR CC for exp
c_h2_gen_stst = ['H2 Electrolysis', 'SMR CC']
c_h2_gen_exp = ['H2 Electrolysis']
c_h2_gen = ['H2 Electrolysis', 'SMR CC']
c_h2_con = ['H2 liquefaction', 'Sabatier', 'Fischer-Tropsch']

In [ ]:
# Network imports
stst = pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")
exp = pypsa.Network("../data/raw/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc")

In [ ]:
# stst and exp dataframes
# spatial
df_stst_ons = pd.read_pickle("../data/processed/df_stst_ons.pkl")
df_stst_off = pd.read_pickle("../data/processed/df_stst_off.pkl")
df_exp_ons = pd.read_pickle("../data/processed/df_exp_ons.pkl")
df_exp_off = pd.read_pickle("../data/processed/df_exp_off.pkl")

# temporal
df_stst_ts = pd.read_pickle("../data/processed/df_stst_ts.pkl")
df_exp_ts = pd.read_pickle("../data/processed/df_exp_ts.pkl")

In [ ]:
# Notebook Functions

def gini(x):
    # (Warning: This is a concise implementation, but it is O(n**2)
    # in time and memory, where n = len(x).  *Don't* pass in huge
    # samples!)

    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad/np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g


In [ ]:
#assert 0

## General

#### MV across regions

In [ ]:
# Further investigation:
df_stst_ons[[c + "_mv_gen_h2" for c in c_h2_gen_stst]].mean().mean() # 120
df_exp_ons[[c + "_mv_gen_h2" for c in c_h2_gen_exp]].mean().mean() # 81

In [ ]:
# delete mv, vf for SMR CC in exp
df_exp_ons["SMR CC_mv_gen_h2_qt"] = pd.DataFrame(np.nan, index=df_exp_ons.index, columns = ["SMR CC_mv_gen_h2"])
df_exp_ons["SMR CC_vf_gen_h2_qt"] = pd.DataFrame(np.nan, index=df_exp_ons.index, columns = ["SMR CC_mv_gen_h2"])
df_exp_ons["SMR CC_vf_dw_gen_h2_qt"] = pd.DataFrame(np.nan, index=df_exp_ons.index, columns = ["SMR CC_mv_gen_h2"])

In [ ]:
# hydrogen generating and consuming

# data
c_h2_con_index = c_h2_con #[x for x in c_h2_con if x != 'H2 liquefaction']
mv_index = [c+ "_mv_gen_h2_qt" for c in c_h2_gen] + [c + "_mv_con_h2_qt" for c in c_h2_con_index]
gen_con_index = [c+ "_gen_h2" for c in c_h2_gen] + [c + "_con_h2" for c in c_h2_con_index]
index = c_h2_gen + c_h2_con_index
ticks = [carrier_renaming.get(n, n) for n in index]

stst_h2_gen_con = df_stst_ons[mv_index]
stst_h2_gen_con = stst_h2_gen_con.values
exp_h2_gen_con = df_exp_ons[mv_index]
exp_h2_gen_con = exp_h2_gen_con.values

# boxplot properties
medianprops = dict(color="black",linewidth=1.5)
meanprops = {"marker":"d","markerfacecolor":"white", "markeredgecolor":"black"}
flierprops= {'marker': 'x', 'markersize': 5, 'markeredgecolor': 'black'}
whiskerprops = dict(linestyle='-',linewidth=1.0, color='black')

# Filter data using np.isnan
mask_stst = ~np.isnan(stst_h2_gen_con)
filtered_stst = [d[m] for d, m in zip(stst_h2_gen_con.T, mask_stst.T)]
mask_exp = ~np.isnan(exp_h2_gen_con)
filtered_exp = [d[m] for d, m in zip(exp_h2_gen_con.T, mask_exp.T)]

fig, ax = plt.subplots(figsize=(12, 6))

stst_plot = plt.boxplot(filtered_stst,
                        positions=np.array(np.arange(len(ticks)))*2.0-0.35,
                        widths=0.6,
                        patch_artist=True,
                        showmeans=True,
                        meanprops=meanprops,
                        medianprops=medianprops,
                        flierprops=flierprops,
                        whiskerprops=whiskerprops
                        )

exp_plot = plt.boxplot(filtered_exp,
                       positions=np.array(np.arange(len(ticks)))*2.0+0.35,
                       widths=0.6,
                       patch_artist=True,
                       showmeans=True,
                       meanprops=meanprops,
                       medianprops=medianprops,
                       flierprops=flierprops,
                       whiskerprops=whiskerprops
                       )


# generation weighted average
wa_mv_stst =np.multiply(df_stst_ons[mv_index], (df_stst_ons[gen_con_index] / df_stst_ons[gen_con_index].sum())).sum()
wa_mv_exp =np.multiply(df_exp_ons[mv_index], (df_exp_ons[gen_con_index] / df_exp_ons[gen_con_index].sum())).sum()
# set value for SMR CC in exp to nan
wa_mv_exp["SMR CC_mv_gen_h2_qt"] = np.nan

ax.plot(np.array(np.arange(len(ticks)))*2.0-0.35, wa_mv_stst.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white", zorder=3)
ax.plot(np.array(np.arange(len(ticks)))*2.0+0.35, wa_mv_exp.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white",zorder=4)


for box, col in zip(stst_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)
    box.set_linestyle('--')

for box, col in zip(exp_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)

# sample sizes
for i, sample_size in enumerate(df_stst_ons[mv_index].count()):
    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
        xytext=((i+0.35)/len(index),1), textcoords='axes fraction', color="blue")

#for i, sample_size in enumerate(df_exp_ons[mv_index].count()):
#    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
#        xytext=((i+0.55)/len(index),1), textcoords='axes fraction', color="red")

# explanations
plt.xticks(np.arange(0, len(ticks) * 2, 2), ticks)
plt.ylabel("market value [$€/MWh_{h2}$]")
plt.xticks(rotation=90)
# plt.title("Market values of electricity generating technologies across the regions (STST vs. EXP)", fontsize=16, pad=20,  **font1)

# cosmetics
ax.patch.set_facecolor('lightgrey')
ax.patch.set_alpha(0.5)

# horizontal lines
ax.axhline(df_stst_ons[mv_index].mean().mean(), ls="--", c='black',linewidth=0.5)
ax.axhline(df_exp_ons[mv_index].mean().mean(), ls="-", c='black', linewidth=0.5)

# legend
patch1 = matplotlib.patches.Patch(ls="--", facecolor="white", edgecolor="black")
patch2 = matplotlib.patches.Patch(ls="-", facecolor="white", edgecolor="black")
line1 = Line2D([0], [0], ls="--", c='black', label="mean (STST)", linewidth=0.5)
line2 = Line2D([0], [0], ls="-", c='black', label="mean (EXP)", linewidth=0.5)
ax.legend([patch1, patch2, line1, line2], ['STST', 'EXP', 'mean (STST)','mean (EXP)'])

fig.tight_layout()
plt.show()

fig.savefig(f"{PLOT_DIR}/mv_h2_gen_con_box.png")

In [ ]:
wa_mv_stst

In [ ]:
wa_mv_exp

In [ ]:
c = "Sabatier"
df_stst_ons[[f"{c}_mv_con_h2_qt", f"{c}_con_h2"]].dropna().sort_values(by=f"{c}_con_h2")
#df_exp_ons[[f"{c}_mv_con_h2_qt", f"{c}_con_h2"]].dropna().sort_values(by=f"{c}_con_h2")

In [ ]:
df_stst_ons[mv_index].describe()

In [ ]:
df_exp_ons[mv_index].describe()

## Spatial Differences

### Spatial correlation of mv, gen

In [ ]:
# Spatial correlation of mv

fig, axs = plt.subplots(ncols=2, figsize=(15, 5))

for i, df, ax in zip([0,1], [df_stst_ons, df_exp_ons], axs):

    corr = df[[c + "_mv" for c in c_el_gen_s]].corr()
    corr.index = [carrier_renaming.get(n, n) for n in c_el_gen_s]
    corr.columns = [carrier_renaming.get(n, n) for n in c_el_gen_s]
    mask = np.triu(np.ones_like(corr, dtype=bool))

    sns.heatmap(corr[abs(corr) > 0.4], mask=mask, cmap="magma_r", annot=True, ax=ax)
    ax.set_title('STST' if i == 0 else 'EXP')

fig.tight_layout(pad=1.5)
plt.close()
plt.show()

#fig.savefig(f"{PLOT_DIR}/mv_el_gen_both_corr.png")

In [ ]:
# Spatial correlation of purchasing prices

fig, axs = plt.subplots(ncols=2, figsize=(8, 3))

for df, ax in zip([df_stst_ons, df_exp_ons], axs):
    corr = df[[c + "_cost_mv_el" for c in c_el_con_s]].corr()
    corr.index = [carrier_renaming.get(n, n) for n in c_el_con_s]
    corr.columns = [carrier_renaming.get(n, n) for n in c_el_con_s]

    mask = np.triu(np.ones_like(corr, dtype=bool))

    sns.heatmap(corr[abs(corr) > 0.5], mask=mask, cmap="magma_r", annot=True, ax=ax)
fig.tight_layout(pad=1.5)

plt.close()
plt.show()

In [ ]:
# Spatial correlation of both

fig, axs = plt.subplots(ncols=2, figsize=(17, 6))

for df, ax in zip([df_stst_ons, df_exp_ons], axs):
    corr = df[[c + "_mv" for c in c_el_gen_s] + [c + "_cost_mv_el" for c in c_el_con_s]].corr()
    corr.index = [carrier_renaming.get(n, n) for n in c_el_gen_s] + [carrier_renaming.get(n, n) + " (con)" for n in c_el_con_s]
    corr.columns = [carrier_renaming.get(n, n) for n in c_el_gen_s] + [carrier_renaming.get(n, n)+ " (con)" for n in c_el_con_s]

    mask = np.triu(np.ones_like(corr, dtype=bool))

    sns.heatmap(corr[abs(corr) > 0.5], mask=mask, cmap="magma_r", annot=True, ax=ax)
fig.tight_layout(pad=1.5)
plt.close()
plt.show()

In [ ]:
# correlation analysis
# what do you want to find?


corr_gini_res = pd.DataFrame(index=c_el_gen_s, columns=[["corr_mv_gen_STST", "cor_mv_gen_EXP", "gini_gen_STST", "gini_gen_EXP", "gini_gen_diff_EXP-STST", "gini_mv_STST", "gini_mv_EXP", "gini_mv_diff_EXP-STST"]])
for c in c_el_gen_s:
    corr_gini_res.loc[c,"corr_mv_gen_STST"] = df_stst_ons[f"{c}_mv"].corr(df_stst_ons[f"{c}_gen"])
    corr_gini_res.loc[c,"cor_mv_gen_EXP"] = df_exp_ons[f"{c}_mv"].corr(df_exp_ons[f"{c}_gen"])
    corr_gini_res.loc[c,"gini_gen_STST"] = gini(df_stst_ons[[f"{c}_gen"]].dropna().values)
    corr_gini_res.loc[c,"gini_gen_EXP"] = gini(df_exp_ons[[f"{c}_gen"]].dropna().values)
    corr_gini_res.loc[c,"gini_gen_EXP"] = gini(df_exp_ons[[f"{c}_gen"]].dropna().values)
    corr_gini_res.loc[c,"gini_mv_STST"] = gini(df_stst_ons[[f"{c}_mv"]].dropna().values)
    corr_gini_res.loc[c,"gini_mv_EXP"] = gini(df_exp_ons[[f"{c}_mv"]].dropna().values)

corr_gini_res["gini_gen_diff_EXP-STST"] = corr_gini_res["gini_gen_EXP"].values - corr_gini_res["gini_gen_STST"].values
corr_gini_res["gini_mv_diff_EXP-STST"] = corr_gini_res["gini_mv_EXP"].values - corr_gini_res["gini_mv_STST"].values
corr_gini_res

**Findings:**
- gini: the higher, the more uneven distributed
- generation is more uneven distributed in EXP case
- mv are more evenly distributed in EXP case

### Spatial map

In [ ]:
# hydrogen consuming and generating: market values across regions as map

model = "EXP"
c_h2_con_index = c_h2_con

if model == "STST":
    df = df_stst_ons
    c_h2_gen_index = c_h2_gen_stst

elif model == "EXP":
    df = df_exp_ons
    c_h2_gen_index = c_h2_gen_exp

index = c_h2_gen_index + c_h2_con
mv_index = [c+ "_mv_gen_h2_qt" for c in c_h2_gen_index] + [c + "_mv_con_h2_qt" for c in c_h2_con_index]
gen_con_index = [c+ "_gen_h2" for c in c_h2_gen_index] + [c + "_con_h2" for c in c_h2_con_index]
ticks = [carrier_renaming.get(n, n) for n in index]

fig, axs = plt.subplots(ncols=2, nrows=math.ceil(len(index)/2), subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(s_w, math.ceil(len(index)/2)*s_h))
crs = ccrs.EqualEarth()

# align colorbar for comparison
mvs = df[mv_index]
vmin = np.nanmin(mvs)
vmax = np.nanmax(mvs)

for i, ax in enumerate(axs.reshape(-1)):

    if i >= len(index):
        ax.axis('off')
        continue

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=mv_index[i],
                                   ax=ax,
                                   cmap=plt.get_cmap("magma_r"),
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   #vmin=vmin,
                                   #vmax=vmax,
                                   legend_kwds={'label':"market values ($€/MWh_{h2}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )

    max_size = df[gen_con_index[i]].abs().max()
    # blue if negative and green if positive
    colors = ['darkblue' if (x < 0) else 'darkgreen' for x in df[gen_con_index[i]] ]

    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(df[gen_con_index[i]].abs() / max_size) * 300,  color=colors, edgecolor="white")
    circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="darkgreen", markeredgecolor="white", markersize=10)
    circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="darkblue", markeredgecolor="white", markersize=10)
    circle3 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

    unit = "$TWh_{h2}$" if max_size > 1e3 else "$GWh_{h2}$"
    max_size = max_size / 1e3 if max_size > 1e3 else max_size
    ax.legend((circle1, circle2, circle3), ('Generation', 'Consumption', f"max circle size:\n{round(max_size)} {unit}"), numpoints=1, loc="upper left", prop={'size': 9})

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{ticks[i]}", fontsize=16, **font1)

# fig.suptitle(f"Spatial Differences in the electricity generation of the VRE technologies ({model})", fontsize=16, **font1)
fig.tight_layout()

#plt.close()
plt.show()

fig.savefig(f"{PLOT_DIR}/mv_h2_gen_con_map_{model}.png")

In [ ]:
#dfdf = df_stst_ons[f"{mv_index[i]}_STST-EXP"].dropna().sort_values()
#i_de = dfdf.index[dfdf.index.str.contains("DE")]

In [ ]:
i = 3
df_stst_ons[[mv_index[i], gen_con_index[i]]].dropna().sort_values(ascending=False, by=mv_index[i])
df_exp_ons[[mv_index[i], gen_con_index[i]]].dropna().sort_values(ascending=False, by=mv_index[i])

In [ ]:
#i = 3
#df_stst_ons[f"{mv_index[i]}_STST-EXP"].dropna().sort_values()#.loc[["GB5 16", "GB5 12", "GB5 0", "GB4 0"]]#.mean()

In [ ]:
# Further investigation
# correlation
vf_index = [c+ "_vf_gen_h2_qt" for c in c_h2_gen_index] + [c + "_vf_con_h2_qt" for c in c_h2_con_index]
cf_index = [c+ "_cf_gen_h2" for c in c_h2_gen_index] + [c + "_cf_con_h2" for c in c_h2_con_index]

for mv, gen_con, vf, cf in zip(mv_index, gen_con_index, vf_index, cf_index):
    print(mv)
    print(df_stst_ons[mv].corr(df_stst_ons[gen_con]))
    print(df_stst_ons[mv].corr(df_stst_ons[vf]))
    print(df_stst_ons[mv].corr(df_stst_ons[cf]))

In [ ]:
#EXP

for mv, gen_con, vf, cf in zip(mv_index, gen_con_index, vf_index, cf_index):
    print(mv)
    print(df_exp_ons[mv].corr(df_exp_ons[gen_con]))
    print(df_exp_ons[mv].corr(df_exp_ons[vf]))
    print(df_exp_ons[mv].corr(df_exp_ons[cf]))

In [ ]:
# test Lorenz kind of

for mv, gen_con in zip(mv_index, gen_con_index):
    df_test = df_stst_ons[[mv, gen_con]].dropna().sort_values(ascending=True, by=mv)
    plt.plot(abs(df_test[gen_con].cumsum() / df_test[gen_con].sum()), df_test[mv], label=mv)
plt.legend()
plt.show()

In [ ]:
# difference in mv

c_h2_gen_index = c_h2_gen_exp
c_h2_con_index = c_h2_con

index = c_h2_gen_index + c_h2_con_index
mv_index = [c+ "_mv_gen_h2_qt" for c in c_h2_gen_index] + [c + "_mv_con_h2_qt" for c in c_h2_con_index]
gen_con_index = [c+ "_gen_h2" for c in c_h2_gen_index] + [c + "_con_h2" for c in c_h2_con_index]
ticks = [carrier_renaming.get(n, n) for n in index]

# STST - EXP
for c in mv_index:
    df_stst_ons[f"{c}_mv_qt_STST-EXP"] = df_stst_ons[c] - df_exp_ons[c]

fig, axs = plt.subplots(ncols=2, nrows=math.ceil(len(index)/2), subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(s_w, math.ceil(len(index)/2) * s_h))
crs = ccrs.EqualEarth()

# align colorbar for comparison
mvs = df_stst_ons[[c + "_mv_qt_STST-EXP" for c in mv_index]]
abs_max_total = max(abs(np.nanmax(mvs)) , abs(np.nanmin(mvs)))

for i, ax in enumerate(axs.reshape(-1)):

    if i >= len(index):
        ax.axis('off')
        continue

    abs_max = df_stst_ons[f"{mv_index[i]}_mv_qt_STST-EXP"].abs().max()
    #unit_cap = "TW" if abs_max > 1e4 else "GW"
    #df[f"{carriers[i]}_cap_STST-EXP"] = (df[f"{carriers[i]}_cap_STST-EXP"] / 1e3) if abs_max > 1e4 else df[f"{carriers[i]}_cap_STST-EXP"]
    #abs_max = abs_max / 1e3 if abs_max > 1e4 else abs_max

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df_stst_ons.to_crs(crs.proj4_init).plot(column=f"{mv_index[i]}_mv_qt_STST-EXP",
                                   ax=ax,
                                   cmap=plt.get_cmap('RdYlGn'),
                                   vmax=abs_max_total,
                                   vmin=-abs_max_total,
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"market value difference ($€/MWh_{h2}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )
    if ticks[i] != "H2 liquefaction":
        # difference in generation (STST-EXP)
        gen_diff = df_stst_ons[gen_con_index[i]].abs() - df_exp_ons[gen_con_index[i]].abs()
        # red if negative and green if positive
        colors = ['red' if (x < 0) else 'green' for x in gen_diff ]

        max_size = abs(gen_diff).max()
        df_stst_ons.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(abs(gen_diff) / max_size) * 300,  color=colors, edgecolor="white")
        circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="green", markeredgecolor="white", markersize=10)
        circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="red", markeredgecolor="white", markersize=10)
        circle3 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

        unit = "$TWh_{h2}$" if max_size > 1e3 else "$GWh_{h2}$"
        max_size = max_size / 1e3 if max_size > 1e3 else max_size
        ax.legend((circle1, circle2, circle3), ('Increased gen /\ncon in STST', 'Increased gen /\ncon in EXP', f"max circle size:\n {round(max_size)} {unit}"), numpoints=1, loc="upper left")


    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{ticks[i]} market values (STST - EXP)", fontsize=16, **font1)

fig.tight_layout(pad=3)

#plt.close()
plt.show()

fig.savefig(f"{PLOT_DIR}/h2_mv_STST-EXP_map_all.png")

In [ ]:
# Further investigation
c = "onwind"
#df_stst_ons[f"{c}_mv_qt_STST-EXP"].dropna().sort_values()

In [ ]:
#de_i = df_stst_ons[f"{c}_mv_qt_STST-EXP"].dropna().index.str.contains("BE")
#df_stst_ons[f"{c}_mv_qt_STST-EXP"].dropna().loc[de_i].mean()

### Value Factor Analysis

In [ ]:
df_stst_ons.columns[df_stst_ons.columns.str.contains("H2_lmp")]

In [ ]:
df_stst_ons[[f"{c}_vf_dw_gen_h2" for c in c_h2_gen_stst]]

In [ ]:
# hydrogen producing and consuming technologies

# data
c_h2_con_index = [x for x in c_h2_con if x != 'H2 liquefaction'] # c_h2_con #
vf_index = [c+ "_vf_dw_gen_h2_qt" for c in c_h2_gen] + [c + "_vf_dw_con_h2_qt" for c in c_h2_con_index]
gen_con_index = [c+ "_gen_h2" for c in c_h2_gen] + [c + "_con_h2" for c in c_h2_con_index]
index = c_h2_gen + c_h2_con_index
ticks = [carrier_renaming.get(n, n) for n in index]

stst_h2_gen_con = df_stst_ons[vf_index]
stst_h2_gen_con = stst_h2_gen_con.values
exp_h2_gen_con = df_exp_ons[vf_index]
exp_h2_gen_con = exp_h2_gen_con.values

# boxplot properties
medianprops = dict(color="black",linewidth=1.5)
meanprops = {"marker":"d","markerfacecolor":"white", "markeredgecolor":"black"}
flierprops= {'marker': 'x', 'markersize': 5, 'markeredgecolor': 'black'}
whiskerprops = dict(linestyle='-',linewidth=1.0, color='black')

# Filter data using np.isnan
mask_stst = ~np.isnan(stst_h2_gen_con)
filtered_stst = [d[m] for d, m in zip(stst_h2_gen_con.T, mask_stst.T)]
mask_exp = ~np.isnan(exp_h2_gen_con)
filtered_exp = [d[m] for d, m in zip(exp_h2_gen_con.T, mask_exp.T)]

fig, ax = plt.subplots(figsize=(12, 6))

stst_plot = plt.boxplot(filtered_stst,
                        positions=np.array(np.arange(len(ticks)))*2.0-0.35,
                        widths=0.6,
                        patch_artist=True,
                        showmeans=True,
                        meanprops=meanprops,
                        medianprops=medianprops,
                        flierprops=flierprops,
                        whiskerprops=whiskerprops
                        )

exp_plot = plt.boxplot(filtered_exp,
                       positions=np.array(np.arange(len(ticks)))*2.0+0.35,
                       widths=0.6,
                       patch_artist=True,
                       showmeans=True,
                       meanprops=meanprops,
                       medianprops=medianprops,
                       flierprops=flierprops,
                       whiskerprops=whiskerprops
                       )

# generation / consumption weighted average
wa_vf_stst =np.multiply(df_stst_ons[vf_index], (abs(df_stst_ons[gen_con_index]) / abs(df_stst_ons[gen_con_index]).sum())).sum()
wa_vf_exp =np.multiply(df_exp_ons[vf_index], (abs(df_exp_ons[gen_con_index]) / abs(df_exp_ons[gen_con_index]).sum())).sum()
# set value for SMR CC in exp to nan
wa_vf_exp["SMR CC_vf_dw_gen_h2_qt"] = np.nan

ax.plot(np.array(np.arange(len(ticks)))*2.0-0.35, wa_vf_stst.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white", zorder=3)
ax.plot(np.array(np.arange(len(ticks)))*2.0+0.35, wa_vf_exp.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white",zorder=4)

for box, col in zip(stst_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)
    box.set_linestyle('--')

for box, col in zip(exp_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)

# sample sizes
for i, sample_size in enumerate(df_stst_ons[vf_index].count()):
    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
        xytext=((i+0.4)/len(index),1), textcoords='axes fraction', color="blue")

#for i, sample_size in enumerate(df_exp_ons[vf_index].count()):
#    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
#        xytext=((i+0.55)/len(index),1), textcoords='axes fraction', color="red")

# explanations
plt.xticks(np.arange(0, len(ticks) * 2, 2), ticks)
plt.ylabel("")
plt.xticks(rotation=90)
ax.set_ylabel("value factor [-]")
#plt.title("Value factors of electricity consuming technologies across the regions (STST vs. EXP)", fontsize=16, pad=20,  **font1)

# cosmetics
ax.patch.set_facecolor('lightgrey')
ax.patch.set_alpha(0.5)

# horizontal lines
ax.axhline(df_stst_ons[vf_index].mean().mean(), ls="--", c='black',linewidth=0.5)
ax.axhline(df_exp_ons[vf_index].mean().mean(), ls="-", c='black', linewidth=0.5)

# legend
patch1 = matplotlib.patches.Patch(ls="--", facecolor="white", edgecolor="black")
patch2 = matplotlib.patches.Patch(ls="-", facecolor="white", edgecolor="black")
line1 = Line2D([0], [0], ls="--", c='black', label="mean (STST)", linewidth=0.5)
line2 = Line2D([0], [0], ls="-", c='black', label="mean (EXP)", linewidth=0.5)
ax.legend([patch1, patch2, line1, line2], ['STST', 'EXP', 'mean (STST)','mean (EXP)'], loc="upper left")

fig.tight_layout()
plt.show()

fig.savefig(f"{PLOT_DIR}/vf_h2_gen_con_box.png")

In [ ]:
wa_vf_stst.sort_values()

In [ ]:
wa_vf_exp.sort_values()

In [ ]:
wa_vf_stst - wa_vf_exp

In [ ]:
df_stst_ons[vf_index].describe()

In [ ]:
# lmps for H2 liquid are the same for all time steps but different in every region
stst.buses_t.marginal_price.loc[:, stst.buses.carrier == "H2 liquid"]

### Case Study: H2 Comparison
comparison of wind driven baltic sea regions and sun driven Mediterranean Sea regions

In [ ]:
# STST

df = df_stst_ons
help = df[df_stst_ons["H2 Electrolysis_gen_h2"] > np.quantile(df["H2 Electrolysis_gen_h2"], 0.5)][["name", "coords"]]
help['x'] = [x[0] for x in help.coords]
help['y'] = [x[1] for x in help.coords]
north_stst = help[help['y'] >= 45].index
south_stst = help[help['y'] < 45].index

In [ ]:
len(north_stst)

In [ ]:
len(south_stst)

In [ ]:
df.loc[north_stst.tolist() +south_stst.tolist(), "H2 Electrolysis_gen_h2"].sum() / df.loc[:, "H2 Electrolysis_gen_h2"].sum()

In [ ]:
df.loc[north_stst.tolist(), "H2 Electrolysis_gen_h2"].sum() / df.loc[:, "H2 Electrolysis_gen_h2"].sum()

In [ ]:
df.loc[south_stst.tolist(), "H2 Electrolysis_gen_h2"].sum() / df.loc[:, "H2 Electrolysis_gen_h2"].sum()

In [ ]:
# corr
# spatial north
df_stst_ons.loc[north_stst, "H2 Electrolysis_gen_h2"].corr(df_stst_ons.loc[north_stst, "solar_gen_el"]) # -0.0284
df_stst_ons.loc[north_stst, "H2 Electrolysis_gen_h2"].corr(df_stst_ons.loc[north_stst, "onwind_gen_el"]) # 0.8670

In [ ]:
#spatial south
df_stst_ons.loc[south_stst, "H2 Electrolysis_gen_h2"].corr(df_stst_ons.loc[south_stst, "solar_gen_el"]) # 0.66985
df_stst_ons.loc[south_stst, "H2 Electrolysis_gen_h2"].corr(df_stst_ons.loc[south_stst, "onwind_gen_el"]) # 0.39617

In [ ]:
df = df_exp_ons
help = df[df["H2 Electrolysis_gen_h2"] > np.quantile(df["H2 Electrolysis_gen_h2"], 0.5)][["name", "coords"]]
help['x'] = [x[0] for x in help.coords]
help['y'] = [x[1] for x in help.coords]
north_exp = help[help['y'] >= 45].index
south_exp = help[help['y'] < 45].index

In [ ]:
len(north_exp)

In [ ]:
len(south_exp)

In [ ]:
df.loc[north_exp.tolist() + south_exp.tolist(), "H2 Electrolysis_gen_h2"].sum() / df.loc[:, "H2 Electrolysis_gen_h2"].sum()

In [ ]:
df.loc[north_exp.tolist(), "H2 Electrolysis_gen_h2"].sum() / df.loc[:, "H2 Electrolysis_gen_h2"].sum()

In [ ]:
df.loc[south_exp.tolist(), "H2 Electrolysis_gen_h2"].sum() / df.loc[:, "H2 Electrolysis_gen_h2"].sum()

In [ ]:
# corr
# spatial north
df_exp_ons.loc[north_exp, "H2 Electrolysis_gen_h2"].corr(df_exp_ons.loc[north_exp, "solar_gen_el"])  # -0.04
#df_exp_ons.loc[north_exp, "H2 Electrolysis_gen_h2"].corr(df_exp_ons.loc[north_exp, "onwind_gen_el"])  # 0.86


In [ ]:
#spatial south
df_exp_ons.loc[south_exp, "H2 Electrolysis_gen_h2"].corr(df_exp_ons.loc[south_exp, "solar_gen_el"])  # 0.76
#df_exp_ons.loc[south_exp, "H2 Electrolysis_gen_h2"].corr(df_exp_ons.loc[south_exp, "onwind_gen_el"])  # 0.65

In [ ]:
c = "H2"
models = ["STST", "STST", "EXP", "EXP"]
regions = [north_stst, south_stst, north_exp, south_exp]
regions_name = ["north", "south", "north", "south"]
df_res = pd.DataFrame(index=stst.buses_t.marginal_price.index.hour.unique()[::-1])

fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(16, 9))

for i, ax in enumerate(axs.reshape(-1)):

    if models[i] == "STST":
        n = stst
    elif models[i] == "EXP":
        n = exp

    buses = n.buses[n.buses.carrier == c].index
    help_df = pd.DataFrame(n.buses.loc[buses]["location"])
    help_df["bus"] = help_df.index
    help_df.set_index('location', inplace=True)

    # change to mean for investigating in mean (unit: €/MWh)
    df = pd.DataFrame(n.buses_t.marginal_price[pd.Index(help_df.loc[regions[i]].bus)].mean(axis=1))

    hours = df.index.hour.unique()[::-1]
    df_start = pd.DataFrame(index=pd.Index(df.index.date).unique())

    for hour in hours:
        df_start[str(hour)] = df[df.index.hour==hour].values

    sns.heatmap(df_start.transpose(),
                ax=ax,
                     cmap=plt.get_cmap("magma_r"),
                     linewidth=0.001,
                     xticklabels=15)
    ax.set_title(f"{c} lmps ({regions_name[i]}, {models[i]})", fontsize=16, **font1)
    ax.set_ylabel("hour of the day", fontsize=12, **font1)
    ax.set_xlabel("day of the year", fontsize=12, **font1)

    # Rewrite the y labels
    x_labels = ax.get_xticks()
    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%d-%b'))

    # quantify effect
    df_res[f"{models[i]}_{regions_name[i]}"] = (df_start.T - df_start.max(axis=1)).mean(axis=1).values

fig.tight_layout(pad=2)
plt.show()

fig.savefig(f"{PLOT_DIR}/case_north_south_lmps_h2.png")

In [ ]:
# quantify effect
df_res

In [ ]:
df_res["STST_north"] - df_res["STST_south"]

In [ ]:
df_res["EXP_north"] - df_res["EXP_south"]

### Congestion rent
Additionally: Calculate congestion rents for link components: price difference i-j * flow i-j
ToDo: maybe also for line components?

In [ ]:
stst.links[stst.links.carrier == "H2 Electrolysis"]

In [ ]:
# congestion rent makes only sense if link has only 2 buses; definition of cr is otherwise unclear
n = stst
carriers = set(n.links.carrier[n.links.bus3 == ""].unique().tolist()).intersection(c_h2_gen+c_h2_con+c_el_gen_s+c_el_con_s)
cr_res1 = pd.DataFrame(index = range(1), columns=carriers)

for carrier in carriers:
    cr_res1[carrier] = (congestion_rent_link(n,carrier).sum().sum() / 1e6)
cr_res1

In [ ]:
n = stst
carrier = "H2 Electrolysis"

crs = []
crs_norm = []

buses = ["bus0", "bus1", "bus2", "bus3", "bus4"]
ps = ["p0", "p1", "p2", "p3", "p4"]
cr = 0

for bus, p in zip(buses, ps):
    if n.links[n.links.carrier == carrier][bus][0] != "":
        cr -= n.links_t[p].loc[:, n.links.carrier == carrier].multiply(
            n.buses_t.marginal_price[n.links[bus][n.links.carrier == carrier]].values)
        crs.append(n.links_t[p].loc[:, n.links.carrier == carrier].multiply(
            n.buses_t.marginal_price[n.links[bus][n.links.carrier == carrier]].values))


In [ ]:
for n, df in zip([stst, exp], [df_stst_ons, df_exp_ons]):

    # electricity prices
    #n.buses_t.marginal_price[n.links["bus0"][n.links.carrier == carrier]]

    # electricity amounts
    # #n.links_t["p0"].loc[:, n.links.carrier == carrier]

    # hydrogen prices
    #n.buses_t.marginal_price[n.links["bus1"][n.links.carrier == carrier]]

    # hydrogen amounts
    #n.links_t["p1"].loc[:, n.links.carrier == carrier] * -1

    # cr
    # revenue hydrogen
    rh = abs(n.buses_t.marginal_price[n.links["bus1"][n.links.carrier == carrier]].values * n.links_t["p1"].loc[:, n.links.carrier == carrier].values)

    # cost electricity
    ce = abs(n.buses_t.marginal_price[n.links["bus0"][n.links.carrier == carrier]].values * (n.links_t["p0"].loc[:, n.links.carrier == carrier] * -1))

    # cr [€]
    (rh - ce).sum().sum() / 1e6

    # normalised [€/MWh_el]
    (rh - ce).sum() / abs(n.links_t["p0"].loc[:, n.links.carrier == carrier].sum())

    # normalised [€/MWh_h2]
    cr_electrolysis_norm_h2 = (rh - ce).sum() / abs(n.links_t["p1"].loc[:, n.links.carrier == carrier].sum())
    cr_electrolysis_norm_h2.index = cr_electrolysis_norm_h2.index.map(n.links.bus1).map(n.buses.location)
    df["H2 Electrolysis_cr_norm_h2"] = cr_electrolysis_norm_h2

    # total amount per region
    cr = congestion_rent_link(n,carrier).sum()
    cr.index = cr.index.map(n.links.bus1).map(n.buses.location)
    # in million €
    df["H2 Electrolysis_cr"] = cr / 1e6

In [ ]:
# billion €
df_stst_ons["H2 Electrolysis_cr"].sum() / 1e3

In [ ]:
df_exp_ons["H2 Electrolysis_cr"].sum() / 1e3

In [ ]:
# mean
df_stst_ons["H2 Electrolysis_cr_norm_h2"].mean() # 22
df_exp_ons["H2 Electrolysis_cr_norm_h2"].mean() # 15.81

In [ ]:
# gwcr
(df_stst_ons["H2 Electrolysis_cr_norm_h2"]*(df_stst_ons["H2 Electrolysis_gen_h2"] / df_stst_ons["H2 Electrolysis_gen_h2"].sum())).sum() # 20.27
#(df_exp_ons["H2 Electrolysis_cr_norm_h2"]*(df_exp_ons["H2 Electrolysis_gen_h2"] / df_exp_ons["H2 Electrolysis_gen_h2"].sum())).sum() # 20.27

In [ ]:
# corr
df_stst_ons[["H2 Electrolysis_cr_norm_h2","H2 Electrolysis_gen_h2"]].corr() # -0.19
df_exp_ons[["H2 Electrolysis_cr_norm_h2","H2 Electrolysis_gen_h2"]].corr() # 0.32
df_stst_ons[["H2 Electrolysis_cr","H2 Electrolysis_gen_h2"]].corr() # -0.98
df_exp_ons[["H2 Electrolysis_cr","H2 Electrolysis_gen_h2"]].corr() # 0.98

In [ ]:
df_stst_ons[["H2 Electrolysis_cr_norm_h2","H2 Electrolysis_gen_h2"]].sort_values(by="H2 Electrolysis_gen_h2", ascending=False)

In [ ]:
df_exp_ons[["H2 Electrolysis_cr_norm_h2","H2 Electrolysis_gen_h2"]].sort_values(by="H2 Electrolysis_cr_norm_h2", ascending=False)

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=1, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(12, 5))
crs = ccrs.EqualEarth()

# align colorbar for comparison
cap1 = df_stst_ons["H2 Electrolysis_cr_norm_h2"]
cap2 = df_exp_ons["H2 Electrolysis_cr_norm_h2"]
vmin = min(np.nanmin(cap1), np.nanmin(cap1))
vmax = max(np.nanmax(cap1), np.nanmax(cap2))

for i, ax in enumerate(axs.reshape(-1)):

    if i == 0:
        model = "STST"
        df = df_stst_ons
    elif i == 1:
        model = "EXP"
        df = df_exp_ons

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column="H2 Electrolysis_cr_norm_h2",
                                   ax=ax,
                                   cmap=plt.get_cmap("magma_r"),
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   vmin=vmin,
                                   vmax=vmax,
                                   legend_kwds={'label':"congestion rent ($€/MWh_{h2}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )

    max_size = df["H2 Electrolysis_cr"].abs().max()

    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(df["H2 Electrolysis_cr"].abs() / max_size) * 200,  color="black", edgecolor="white")
    circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="black", markeredgecolor="black", markersize=10)
    circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

    unit = "million €"
    ax.legend((circle1, circle2), ('Congestion rent',f"max circle size:\n{round(max_size)} {unit}"), numpoints=1, loc="upper left", prop={'size': 9})

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"Electrolysis congestion rent ({model})", fontsize=16, **font1)

# fig.suptitle(f"Spatial Differences in the electricity generation of the VRE technologies ({model})", fontsize=16, **font1)
fig.tight_layout()

#plt.close()
plt.show()

fig.savefig(f"{PLOT_DIR}/cr_electrolysis.png")

In [ ]:
cr_res1 = cr_res1.sort_values(by=0, axis='columns', ascending=False)
cr_res1

In [ ]:
sns.set(rc={'figure.figsize':(20,5)})
sns.barplot(data=cr_res1)
ticks = plt.xticks(rotation=90)
plt.title("congestion rent in million € per year")
plt.show()

__Findings__
- negative values for some technologies: OCGT, water tanks, battery charger but in much smaller magnitude than positive values (maybe due to numeric inaccuracy?)

In [ ]:
# Congestion rent by region
cr_regions_onshore = onshore_regions.copy().rename(columns={"name": "location"})

for carrier in carriers:
    cr_regions = congestion_rent_link(n,carrier).sum()

    if n.buses.loc[cr_regions.index.map(n.links.bus0)].location.values[0] == 'EU':
        bus = "bus1"
    else:
        bus = "bus0"

    cr_regions = pd.concat([cr_regions,n.links.loc[cr_regions.index][bus]], axis=1).rename(columns={0: f"{carrier}_cr"})
    cr_regions["location"] = n.buses.loc[cr_regions.index.map(n.links[bus])].location.values
    # for some carrier e.g. "DC" the locations occur several times in the index -> take sum
    cr_regions = cr_regions.groupby(by=["location"], axis="index").sum()
    # convert to million € per year
    cr_regions_onshore[f"{carrier}_cr"] = (cr_regions[f"{carrier}_cr"] / 1000000)
cr_regions_onshore.head(3)
    # TODO: For plotting the cr into a map which bus should be used? bus0, where the energy comes from or bus1 where the energy goes to? For some carriers e.g. Fischer-Tropsch they are not in the same region (EU bus)

In [ ]:
carrier = "H2 Electrolysis"

fig = plt.figure(figsize=(10, 8))

crs = ccrs.EqualEarth()
ax = plt.axes(projection=ccrs.EqualEarth()) # Mercator
ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
ax.coastlines(edgecolor='black', linewidth=0.5)
ax.set_facecolor('white')
ax.add_feature(cartopy.feature.OCEAN, color='azure')
ax.set_title(f"{carrier} congestion rent", fontsize=16, **font1)

cr_regions_onshore.to_crs(crs.proj4_init).plot(column=f"{carrier}_cr",
                     ax=ax,
                     cmap=plt.get_cmap("magma_r"),
                     linewidth=0.05,
                     edgecolor = 'grey',
                     legend=True,
                     legend_kwds={'label':"million € per year",
                        'orientation': "vertical",
                                  'shrink' : 0.8})

plt.show()
# TODO: Would be interesting to plot a map with the location of the capacity of the corresponding carrier next to it; probably there is a correlation between higher capacity and higher cr

In [ ]:
#cr_regions_onshore[f"{carrier}_cr_norm_h2"].sort_values(ascending=False)

__Findings:__
- __H2 Electrolysis:__  Highest values in IRL, GBR and DNK; General higher values in the coastal area of mainland Europe (PRT, ESP, FRA, NLD, DEU, POL, GRC); very low values in the inner mainland and scandinavia
- __electricity distribution grid:__ no strong pattern; highest values for north/middle (FR0 1) and GB5 13; there might be soe correlation with higher cr in population-intensive regions an
- __Fischer-Tropsch:__ Comparable to H2 Electrolysis; Highest values in DNK & GBR; especialle higher values on the resource receiving party of the european coastline (Atlantic coast and GRC)
- __urban central solid biomass CHP CC:__ High values for parts of DEU, CZE, AUT, ITA, SVK & HUN; Very low values for west europe UK and scandinavia
- residential rural ground heat pump:
- residential urban decentral air heat pump:
- H2 liquefaction:
- urban central air heat pump:
- urban central gas CHP:

### Weighted price differences

In [ ]:
for n, df in zip([stst, exp], [df_stst_ons, df_exp_ons]):
    lmp_h2 = n.buses_t.marginal_price.loc[:, n.buses.carrier == "H2"]
    lmp_h2.columns = lmp_h2.columns.map(n.buses.location)
    lmp_lv = n.buses_t.marginal_price.loc[:, n.buses.carrier == "low voltage"]
    lmp_lv.columns = lmp_lv.columns.map(n.buses.location)
    lmp_el = (n.buses_t.marginal_price.loc[:, n.buses.carrier == "AC"] + lmp_lv) / 2
    lmp_diff = lmp_h2 - lmp_el
    lmp_diff.mean().mean() # STST: 13.53, EXP: 8.23

    # hydrogen generation weighted
    h2_gen = n.links_t.p1.loc[ : , n.links.carrier == "H2 Electrolysis"]
    h2_gen.columns = h2_gen.columns.map(n.links.bus1).map(n.buses.location)

    # overall
    w_all = h2_gen / h2_gen.sum().sum()
    (lmp_diff * w_all).sum().sum()

    # per region
    w_regions = h2_gen / h2_gen.sum()
    df["h2_w_lmp_diff"] = (lmp_diff * w_regions).sum()

In [ ]:
(lmp_diff * w_all).sum().sum()

In [ ]:
df_stst_ons["H2 Electrolysis_cr_norm_h2"]

In [ ]:
df_stst_ons["h2_w_lmp_diff"].describe()

In [ ]:
df_exp_ons["h2_w_lmp_diff"].describe()

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=1, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(12, 5))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    if i == 0:
        model = "STST"
        df = df_stst_ons
    elif i == 1:
        model = "EXP"
        df = df_exp_ons

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column="h2_w_lmp_diff",
                                   ax=ax,
                                   cmap=plt.get_cmap("magma_r"),
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"price difference ($€/MWh_{h2-el}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"Hydrogen generation weighted price difference ({model})", fontsize=16, **font1)

fig.tight_layout()

#plt.close()
plt.show()

### LCOH

![](../../../../Pictures/Screenshots/Screenshot_20221102_182240.png)

![](../../../../Pictures/Screenshots/lcoe_mv.png)

#### MV based

In [ ]:
# MV on country level
df_stst_ons_help = df_stst_ons.copy()
df_stst_ons_help["country_help"] = df_stst_ons_help.index.str[:2]
ccs = [convert_ISO_3166_2_to_1[cc] for cc in df_stst_ons_help["country_help"]]
df_stst_ons_help["country"] = pd.DataFrame([ccs]).T.values

# get mv on country level hydrogen
carriers = ["H2 Electrolysis", "SMR CC"]

df_stst_ons_c_mv = pd.DataFrame(index = df_stst_ons_help["country"].unique())
df_exp_ons_c_mv = pd.DataFrame(index = df_stst_ons_help["country"].unique())

for c in carriers:
    for n, df, df_res in zip([stst, exp],[df_stst_ons, df_exp_ons], [df_stst_ons_c_mv, df_exp_ons_c_mv]):
        g_sum = pd.concat([df[f"{c}_gen_h2"], df_stst_ons_help["country"]], axis=1).groupby(by="country").sum()
        helper = pd.DataFrame(df_stst_ons_help["country"]).merge(g_sum, left_on="country", right_on=g_sum.index, right_index=True)
        helper["result"] = df[f"{c}_mv_gen_h2"] * (df[f"{c}_gen_h2"] / helper[f"{c}_gen_h2"])
        df_res[f"{c}_mv"] = helper.groupby(by="country").sum()["result"]

In [ ]:
# total lcoh by mv of generators (STST)
helper =pd.concat([df_stst_ons["H2 Electrolysis_gen_h2"], df_stst_ons["SMR CC_gen_h2"], df_stst_ons["H2 Electrolysis_gen_h2"] + df_stst_ons["SMR CC_gen_h2"], df_stst_ons_help["country"]],axis=1)
helper = helper.groupby(by="country").sum()
helper["w_electrolysis"] = helper["H2 Electrolysis_gen_h2"] / helper[0]
helper["w_smrcc"] = helper["SMR CC_gen_h2"] / helper[0]
df_stst_ons_c_mv["H2_mv_total"] = helper["w_electrolysis"] * df_stst_ons_c_mv["H2 Electrolysis_mv"] + helper["w_smrcc"] * df_stst_ons_c_mv["SMR CC_mv"]

In [ ]:
# total lcoh on region and country level

# generation weighted lmp per region
c = "H2"

for n, df, df_res in zip([stst, exp],[df_stst_ons, df_exp_ons], [df_stst_ons_c_mv, df_exp_ons_c_mv]):
    nb = nodal_balance(n, carrier=c, time="2013", aggregate=['component'], energy=True)
    nb = nb.unstack(level=[1])
    weights = nb[nb > 0].groupby(by="snapshot").sum().abs()
    lmps = n.buses_t.marginal_price.loc[:, n.buses.carrier == c]
    gw_lmps_h2 = np.multiply(lmps, weights / weights.sum()).sum()
    gw_lmps_h2.index = gw_lmps_h2.index.map(n.buses.location)
    df["gw_lmps_h2"] = gw_lmps_h2

    df[f"{c}_demands"] = weights.sum()
    g_sum = pd.concat([df[f"{c}_demands"], df_stst_ons_help["country"]], axis=1).groupby(by="country").sum()
    helper = pd.DataFrame(df_stst_ons_help["country"]).merge(g_sum, left_on="country", right_on=g_sum.index, right_index=True)
    helper["result"] = df["gw_lmps_h2"] * (df[f"{c}_demands"] / helper[f"{c}_demands"])
    df_res[f"{c}_lmp_dw"] = helper.groupby(by="country").sum()["result"]

# assign scenario (for plot)
df_stst_ons_c_mv = df_stst_ons_c_mv.assign(scenario = "STST")
df_exp_ons_c_mv = df_exp_ons_c_mv.assign(scenario = "EXP")

In [ ]:
df_exp_ons_c_mv.head()

In [ ]:
# Total LCOH over all regions:
# STST
w = pd.concat([df_stst_ons["H2 Electrolysis_gen_h2"] + df_stst_ons["SMR CC_gen_h2"], df_stst_ons_help["country"]],axis=1).groupby(by="country").sum()
w = w / w.sum()
np.multiply(df_stst_ons_c_mv["H2_mv_total"] , w.T).sum(axis=1) # 82.95

# EXP
w = pd.concat([df_exp_ons["H2 Electrolysis_gen_h2"], df_stst_ons_help["country"]],axis=1).groupby(by="country").sum()
w = w / w.sum()
np.multiply(df_exp_ons_c_mv["H2 Electrolysis_mv"] , w.T).sum(axis=1) # 77.59

#### Electrolysis

In [ ]:
df = df_stst_ons
n = stst

# marginal cost
mc = stst.links[stst.links.carrier == "H2 Electrolysis"].marginal_cost
mc.index = mc.index.map(n.links.bus1).map(n.buses.location)
mc = mc * df["H2 Electrolysis_gen_h2"] * 1e3

# capacity investment cost
I = stst.links[stst.links.carrier == "H2 Electrolysis"].capital_cost * stst.links[stst.links.carrier == "H2 Electrolysis"].p_nom_opt
I.index = I.index.map(n.links.bus1).map(n.buses.location)


lcoh = ( I + mc + (df["H2 Electrolysis_mv_con_el"] * df["H2 Electrolysis_con_el"] * 1e3 * -1)) / (df["H2 Electrolysis_gen_h2"] * 1e3)
#lcoh

In [ ]:
# ratio of market values and LCOH of electrolysis
( I + mc + (df["H2 Electrolysis_mv_con_el"] * df["H2 Electrolysis_con_el"] * 1e3 * -1)) / (df["H2 Electrolysis_mv_gen_h2"] * df["H2 Electrolysis_gen_h2"] * 1e3)

In [ ]:
# per country

In [ ]:
# STST
c = "H2 Electrolysis"
# marginal cost
mc = stst.links[stst.links.carrier == c].marginal_cost
mc.index = mc.index.map(n.links.bus1).map(n.buses.location)
mc = mc * df_stst_ons[f"{c}_gen_h2"] * 1e3
df_stst_ons[f"{c}_mc"] = mc

# capacity investment cost
I = stst.links[stst.links.carrier == c].capital_cost * stst.links[stst.links.carrier == c].p_nom_opt
I.index = I.index.map(n.links.bus1).map(n.buses.location)
df_stst_ons[f"{c}_cap_invest"] = I

# plot of generation weighted capacity factors by country
df_stst_ons_c = pd.DataFrame(index = df_stst_ons_help["country"].unique())
df_stst_ons_c[f"{c}_mc"] = df_stst_ons_help.groupby(by="country").sum()[f"{c}_mc"]
df_stst_ons_c[f"{c}_cap_invest"] = df_stst_ons_help.groupby(by="country").sum()[f"{c}_cap_invest"]
df_stst_ons_c[f"{c}_cost_el"] = pd.concat([df_stst_ons_help[f"{c}_mv_con_el"] * df_stst_ons_help[f"{c}_con_el"] * 1e3 * -1 , df_stst_ons_help["country"]], axis=1).groupby(by="country").sum()
df_stst_ons_c[f"{c}_gen_h2"] = df_stst_ons_help.groupby(by="country").sum()[f"{c}_gen_h2"] * 1e3
df_stst_ons_c[f"{c}_lcoh"] = ( df_stst_ons_c[f"{c}_cap_invest"] + df_stst_ons_c[f"{c}_mc"] + df_stst_ons_c[f"{c}_cost_el"]) / df_stst_ons_c[f"{c}_gen_h2"]
df_stst_ons_c = df_stst_ons_c.assign(scenario = "STST")

In [ ]:
# EXP

# marginal cost
mc = exp.links[exp.links.carrier == c].marginal_cost
mc.index = mc.index.map(n.links.bus1).map(n.buses.location)
mc = mc * df_exp_ons[f"{c}_gen_h2"] * 1e3
df_exp_ons[f"{c}_mc"] = mc

# capacity investment cost
I = exp.links[exp.links.carrier == c].capital_cost * exp.links[exp.links.carrier == c].p_nom_opt
I.index = I.index.map(n.links.bus1).map(n.buses.location)
df_exp_ons[f"{c}_cap_invest"] = I

# plot of generation weighted capacity factors by country
df_exp_ons_help = df_exp_ons.copy()
df_exp_ons_help["country_help"] = df_exp_ons_help.index.str[:2]
ccs = [convert_ISO_3166_2_to_1[cc] for cc in df_exp_ons_help["country_help"]]
df_exp_ons_help["country"] = pd.DataFrame([ccs]).T.values

df_exp_ons_c = pd.DataFrame(index = df_exp_ons_help["country"].unique())
df_exp_ons_c[f"{c}_mc"] = df_exp_ons_help.groupby(by="country").sum()[f"{c}_mc"]
df_exp_ons_c[f"{c}_cap_invest"] = df_exp_ons_help.groupby(by="country").sum()[f"{c}_cap_invest"]
df_exp_ons_c[f"{c}_cost_el"] = pd.concat([df_exp_ons_help[f"{c}_mv_con_el"] * df_exp_ons_help[f"{c}_con_el"] * 1e3 * -1 , df_exp_ons_help["country"]], axis=1).groupby(by="country").sum()
df_exp_ons_c[f"{c}_gen_h2"] = df_exp_ons_help.groupby(by="country").sum()[f"{c}_gen_h2"] * 1e3
df_exp_ons_c[f"{c}_lcoh"] = ( df_exp_ons_c[f"{c}_cap_invest"] + df_exp_ons_c[f"{c}_mc"] + df_exp_ons_c[f"{c}_cost_el"]) / df_exp_ons_c[f"{c}_gen_h2"]
df_exp_ons_c = df_exp_ons_c.assign(scenario = "EXP")

In [ ]:
lcohs = pd.concat([df_exp_ons_c[[f"{c}_lcoh", "scenario"]], df_stst_ons_c[[f"{c}_lcoh", "scenario"]]])
lcohs.reset_index(inplace=True)
lcohs.rename(columns={"index": "country"}, inplace=True)
lcohs.pivot('country', 'scenario',f"{c}_lcoh")

In [ ]:
# overall cost / amount
df_stst_ons_c[[f"{c}_mc", f"{c}_cap_invest", f"{c}_cost_el"]].sum().sum() / df_stst_ons_c[f"{c}_gen_h2"].sum() # 78.49
df_exp_ons_c[[f"{c}_mc", f"{c}_cap_invest", f"{c}_cost_el"]].sum().sum() / df_exp_ons_c[f"{c}_gen_h2"].sum() # 78.77

In [ ]:
# according to mv
# STST
w = pd.concat([df_stst_ons["H2 Electrolysis_gen_h2"], df_stst_ons_help["country"]],axis=1).groupby(by="country").sum()
w = w / w.sum()
np.multiply(df_stst_ons_c_mv["H2 Electrolysis_mv"] , w.T).sum(axis=1) # 76.77
# EXP
w = pd.concat([df_exp_ons["H2 Electrolysis_gen_h2"], df_exp_ons_help["country"]],axis=1).groupby(by="country").sum()
w = w / w.sum()
np.multiply(df_exp_ons_c_mv["H2 Electrolysis_mv"] , w.T).sum(axis=1) # 77.59

In [ ]:
df_stst_ons_c[f"{c}_gen_h2"].sum() / 1e6 # 2884.63
#df_exp_ons_c[f"{c}_gen_h2"].sum() / 1e6 # 2419.52

In [ ]:
df_stst_ons_c[f"{c}_lcoh"].describe()

In [ ]:
df_exp_ons_c[f"{c}_lcoh"].describe()

In [ ]:
# cost based

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 5))

lcohs.pivot('country', 'scenario',"H2 Electrolysis_lcoh")[["STST", "EXP"]].plot(kind='bar', ax=ax, color=["cadetblue", "hotpink"], width=0.7)
ax.set_ylabel("LCOH in $€/MWh_{h2}$")
ax.set_title(f"LCOH from electrolysis", fontsize=16, **font1)

plt.show()

In [ ]:
# market value based
c = "H2 Electrolysis"

lcohs_mv = pd.concat([df_exp_ons_c_mv[[f"{c}_mv", "scenario"]], df_stst_ons_c_mv[[f"{c}_mv", "scenario"]]])
lcohs_mv.reset_index(inplace=True)
lcohs_mv.rename(columns={"index": "country"}, inplace=True)
lcohs_mv.pivot('country', 'scenario',f"{c}_mv")

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 5))

lcohs_mv.pivot('country', 'scenario',"H2 Electrolysis_mv")[["STST", "EXP"]].plot(kind='bar', ax=ax, color=["cadetblue", "hotpink"], width=0.7)
ax.set_ylabel("LCOH in $€/MWh_{h2}$")
ax.set_title(f"LCOH from electrolysis per country", fontsize=16, **font1)

plt.tight_layout()
plt.show()
fig.savefig(f"{PLOT_DIR}/lcoh_electrolysis.png")

In [ ]:
#compare to market value
pd.concat([df_stst_ons_c_mv["H2 Electrolysis_mv"],df_stst_ons_c["H2 Electrolysis_lcoh"]],axis=1)

In [ ]:
pd.concat([df_exp_ons_c_mv["H2 Electrolysis_mv"],df_exp_ons_c["H2 Electrolysis_lcoh"]],axis=1)

#### SMR CC

In [ ]:
c = "SMR CC"
stst.links[stst.links.carrier == "SMR CC"]

In [ ]:
# Problem: own Calc does not find cost mv as they are present at EU bus
#df_stst_ons[f"{c}_mv_con_gas"]

In [ ]:
con = stst.links_t.p0.loc[: , stst.links.carrier == c] * 3
lmps = n.buses_t.marginal_price.loc[:, con.columns.map(stst.links.bus0)]
con.columns = con.columns.map(stst.links.bus1).map(stst.buses.location)
lmps.columns = con.columns
cost = con * lmps
smr_cc_cost_gas = cost.sum()

# cost C02 atmosphere
con = stst.links_t.p2.loc[: , stst.links.carrier == c] * 3
lmps = n.buses_t.marginal_price.loc[:, con.columns.map(stst.links.bus2)]
con.columns = con.columns.map(stst.links.bus1).map(stst.buses.location)
lmps.columns = con.columns
cost = con * lmps
smr_cc_cost_co2_atm = cost.sum()

# revenue C02 stored
con = stst.links_t.p3.loc[: , stst.links.carrier == c] * 3
lmps = n.buses_t.marginal_price.loc[:, con.columns.map(stst.links.bus3)]
con.columns = con.columns.map(stst.links.bus1).map(stst.buses.location)
lmps.columns = con.columns
cost = con * lmps
smr_cc_cost_co2_stored = cost.sum()

smr_cc_cost_total = smr_cc_cost_gas + smr_cc_cost_co2_atm + smr_cc_cost_co2_stored

In [ ]:
# STST
c = "SMR CC"
# marginal cost
mc = stst.links[stst.links.carrier == c].marginal_cost
mc.index = mc.index.map(n.links.bus1).map(n.buses.location)
mc = mc * df_stst_ons[f"{c}_gen_h2"] * 1e3
df_stst_ons[f"{c}_mc"] = mc

# capacity investment cost
I = stst.links[stst.links.carrier == c].capital_cost * stst.links[stst.links.carrier == c].p_nom_opt
I.index = I.index.map(n.links.bus1).map(n.buses.location)
df_stst_ons[f"{c}_cap_invest"] = I

df_stst_ons_help = df_stst_ons.copy()
df_stst_ons_help["country_help"] = df_stst_ons_help.index.str[:2]
ccs = [convert_ISO_3166_2_to_1[cc] for cc in df_stst_ons_help["country_help"]]
df_stst_ons_help["country"] = pd.DataFrame([ccs]).T.values

df_stst_ons_c[f"{c}_mc"] = df_stst_ons_help.groupby(by="country").sum()[f"{c}_mc"]
df_stst_ons_c[f"{c}_cap_invest"] = df_stst_ons_help.groupby(by="country").sum()[f"{c}_cap_invest"]
df_stst_ons_c[f"{c}_cost_gas"] = pd.concat([smr_cc_cost_total, df_stst_ons_help["country"]], axis=1).groupby(by="country").sum()
df_stst_ons_c[f"{c}_gen_h2"] = df_stst_ons_help.groupby(by="country").sum()[f"{c}_gen_h2"] * 1e3
df_stst_ons_c[f"{c}_lcoh"] = ( df_stst_ons_c[f"{c}_cap_invest"] + df_stst_ons_c[f"{c}_mc"] + df_stst_ons_c[f"{c}_cost_gas"]) / df_stst_ons_c[f"{c}_gen_h2"]
df_stst_ons_c = df_stst_ons_c.assign(scenario = "STST")

In [ ]:
# ratio of market values and lcoh
lcoh = (smr_cc_cost_total / (df_stst_ons["SMR CC_gen_h2"] * 1e3))

In [ ]:
df_stst_ons["SMR CC_mv_gen_h2"]

In [ ]:
df_stst_ons_c[f"{c}_lcoh"]

In [ ]:
# cost based

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 5))

df_stst_ons_c["SMR CC_lcoh"].plot(kind='bar', ax=ax, color="cadetblue", width=0.6)
ax.set_ylabel("LCOH in $€/MWh_{h2}$")
ax.set_title(f"LCOH from SMR CC per country (STST)", fontsize=16, **font1)

plt.show()

In [ ]:
# mv based

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 5))

df_stst_ons_c_mv["SMR CC_mv"].plot(kind='bar', ax=ax, color="cadetblue", width=0.6)
ax.set_ylabel("LCOH in $€/MWh_{h2}$")
ax.set_title(f"LCOH from SMR CC per country (STST)", fontsize=16, **font1)
ax.set_xlabel("country")

plt.tight_layout()
plt.show()
fig.savefig(f"{PLOT_DIR}/lcoh_smr_cc.png")

In [ ]:
# compare to mv
pd.concat([df_stst_ons_c_mv["SMR CC_mv"],df_stst_ons_c["SMR CC_lcoh"]],axis=1)

In [ ]:
df_stst_ons["SMR CC_mv_gen_h2"]

#### Total

In [ ]:
c1 = "H2 Electrolysis"
c2 = "SMR CC"

df_stst_ons_c["lcoh_total"] = (df_stst_ons_c[f"{c1}_cap_invest"] + df_stst_ons_c[f"{c1}_mc"] + df_stst_ons_c[f"{c1}_cost_el"] + df_stst_ons_c[f"{c2}_cap_invest"] + df_stst_ons_c[f"{c2}_mc"] + df_stst_ons_c[f"{c2}_cost_gas"]) / (df_stst_ons_c[f"{c1}_gen_h2"] + df_stst_ons_c[f"{c2}_gen_h2"])

df_exp_ons_c["lcoh_total"] =  df_exp_ons_c["H2 Electrolysis_lcoh"]

lcohs_total = pd.concat([df_exp_ons_c[["lcoh_total", "scenario"]], df_stst_ons_c[["lcoh_total", "scenario"]]])
lcohs_total.reset_index(inplace=True)
lcohs_total.rename(columns={"index": "country"}, inplace=True)
lcohs_total.pivot('country', 'scenario',"lcoh_total")

In [ ]:
# cost based

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 5))

lcohs_total.pivot('country', 'scenario',"lcoh_total")[["STST", "EXP"]].plot(kind='bar', ax=ax, color=["cadetblue", "hotpink"], width=0.7)
ax.set_ylabel("LCOH in $€/MWh_{h2}$")
ax.set_title(f"Total LCOH per country", fontsize=16, **font1)
ax.legend(loc="upper right")

plt.show()

In [ ]:
lcohs_total.pivot('country', 'scenario',"lcoh_total").describe()

In [ ]:
# market value based
help = df_exp_ons_c_mv[["H2 Electrolysis_mv", "scenario"]]
help.rename(columns={"H2 Electrolysis_mv":"H2_mv_total"}, inplace=True)
lcohs_total_mv = pd.concat([help, df_stst_ons_c_mv[["H2_mv_total", "scenario"]]])
lcohs_total_mv.reset_index(inplace=True)
lcohs_total_mv.rename(columns={"index": "country"}, inplace=True)
lcohs_total_mv.pivot('country', 'scenario',"H2_mv_total")

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 5))

lcohs_total_mv.pivot('country', 'scenario',"H2_mv_total")[["STST", "EXP"]].plot(kind='bar', ax=ax, color=["cadetblue", "hotpink"], width=0.7)
ax.set_ylabel("LCOH in $€/MWh_{h2}$")
ax.set_title(f"LCOH per country", fontsize=16, **font1)
ax.legend(loc="upper right")

plt.tight_layout()
plt.show()
fig.savefig(f"{PLOT_DIR}/lcoh_total.png")

In [ ]:
# compare to demand weighted lmps
pd.concat([df_stst_ons_c_mv["H2_lmp_dw"],df_stst_ons_c_mv["H2_mv_total"], df_stst_ons_c["lcoh_total"]],axis=1)

In [ ]:
df_exp_ons_c_mv

In [ ]:
# compare to demand weighted lmps
pd.concat([df_exp_ons_c_mv["H2_lmp_dw"],df_exp_ons_c_mv["H2 Electrolysis_mv"], df_exp_ons_c["lcoh_total"]],axis=1)

In [ ]:
# total cost per MWh
# cost absed
(df_stst_ons_c[f"{c1}_cap_invest"] + df_stst_ons_c[f"{c1}_mc"] + df_stst_ons_c[f"{c1}_cost_el"] + df_stst_ons_c[f"{c2}_cap_invest"] + df_stst_ons_c[f"{c2}_mc"] + df_stst_ons_c[f"{c2}_cost_gas"]).sum() / (df_stst_ons_c[f"{c1}_gen_h2"] + df_stst_ons_c[f"{c2}_gen_h2"]).sum()

#mv based: 82.95

In [ ]:
# major producers
df_stst_ons_c[["H2 Electrolysis_gen_h2" , "SMR CC_gen_h2"]].sum(axis=1).sort_values(ascending=False)

In [ ]:
df_exp_ons_c["H2 Electrolysis_gen_h2"].sort_values(ascending=False)

In [ ]:
# who produces more in EXP
(df_stst_ons_c[["H2 Electrolysis_gen_h2" , "SMR CC_gen_h2"]].sum(axis=1) - df_exp_ons_c["H2 Electrolysis_gen_h2"]).sort_values()

In [ ]:
(lcohs_total.pivot('country', 'scenario',"lcoh_total")["STST"] - lcohs_total.pivot('country', 'scenario',"lcoh_total")["EXP"]).sort_values()

## Temporal Differences

In [ ]:
df

### Nodal prices per time step

In [ ]:
stst.buses.carrier.unique().tolist()

In [ ]:
carriers = [ "H2", "H2 liquid"]
model = "STST"

if model == "STST":
    n = stst

elif model == "EXP":
    n = exp

fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(16, 4.5))

for i, ax in enumerate(axs):

    # change to mean for investigating in mean (unit: €/MWh)
    df = pd.DataFrame(n.buses_t.marginal_price[n.buses[n.buses.carrier==carriers[i]].index].mean(axis=1))

    hours = df.index.hour.unique()[::-1]
    df_start = pd.DataFrame(index=pd.Index(df.index.date).unique())

    for hour in hours:
        df_start[str(hour)] = df[df.index.hour==hour].values

    sns.heatmap(df_start.transpose(),
                ax =ax,
                cmap=plt.get_cmap("magma_r"),
                linewidth=0.001,
                xticklabels=15,
                cbar_kws={'label': "lmps ($€/MWh_{h2}$)", 'pad': 0.09})
    ax.set_title(f"{carriers[i]}", fontsize=16, **font1)
    ax.set_ylabel("hour of the day", fontsize=12, **font1)
    ax.set_xlabel("day of the year", fontsize=12, **font1)

    # Rewrite the y labels
    x_labels = ax.get_xticks()
    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%d-%b'))

fig.suptitle(f"Locational marginal prices for electricity buses ({model})", fontsize=16, **font1)
fig.tight_layout(pad=1)
plt.show()

fig.savefig(f"{PLOT_DIR}/h2+liquid_lmp_heatmap_{model}.png")

In [ ]:
# average prices
# STST: AC: 92.12; low voltage: 97.55; battery: 93.12; Li-ion: 102.81
# EXP: AC: 75.31; low voltage: 80.74; battery: 75.91; Li-ion: 85.61
c = "AC"
n = exp
n.buses_t.marginal_price.loc[: , n.buses.carrier == c].mean().mean()

In [ ]:
c = "AC"

In [ ]:
from sklearn import preprocessing

sns.distplot(stst.buses_t.marginal_price.loc[: , stst.buses.carrier == c].dropna().values.reshape(1, -1), hist=False, rug=True, label="stst")
sns.distplot(exp.buses_t.marginal_price.loc[: , exp.buses.carrier == c].dropna().values.reshape(1, -1), hist=False, rug=True, label="exp")

plt.legend()
plt.show()

In [ ]:
for c in carriers:
    print(c)
    print(np.std(stst.buses_t.marginal_price.loc[: , stst.buses.carrier == c].values.reshape(1, -1)))
    print(np.std(exp.buses_t.marginal_price.loc[: , exp.buses.carrier == c].values.reshape(1, -1)))

In [ ]:
#stst.buses_t.marginal_price.loc[: , stst.buses.carrier == c].values.reshape(1, -1).sort()
#exp.buses_t.marginal_price.loc[: , exp.buses.carrier == c].values.reshape(1, -1).sort()

np.sort(stst.buses_t.marginal_price.loc[: , stst.buses.carrier == c], axis=None)
np.sort(exp.buses_t.marginal_price.loc[: , exp.buses.carrier == c], axis=None)

In [ ]:
np.std(exp.buses_t.marginal_price.loc[: , exp.buses.carrier == c].values.reshape(1, -1))

In [ ]:
# comparison STST and EXP
carrier = "H2"
models = ["STST", "EXP"]

fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(16, 4.5))

for i, ax in enumerate(axs):

    if models[i] == "STST":
        n = stst

    elif models[i] == "EXP":
        n = exp

    # change to mean for investigating in mean (unit: €/MWh)
    df = pd.DataFrame(n.buses_t.marginal_price[n.buses[n.buses.carrier==carrier].index].mean(axis=1))

    hours = df.index.hour.unique()[::-1]
    df_start = pd.DataFrame(index=pd.Index(df.index.date).unique())

    for hour in hours:
        df_start[str(hour)] = df[df.index.hour==hour].values

    sns.heatmap(df_start.transpose(),
                ax =ax,
                cmap=plt.get_cmap("magma_r"),
                linewidth=0.001,
                xticklabels=15,
                cbar_kws={'label': "lmps ($€/MWh_{h2}$)", 'pad': 0.09})
    ax.set_title(f"H2 ({models[i]})", fontsize=16, **font1)
    ax.set_ylabel("hour of the day", fontsize=12, **font1)
    ax.set_xlabel("day of the year", fontsize=12, **font1)

    # Rewrite the y labels
    x_labels = ax.get_xticks()
    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%d-%b'))

#fig.suptitle(f"Locational marginal prices for electricity buses ({model})", fontsize=16, **font1)
fig.tight_layout(pad=1)
plt.show()

fig.savefig(f"{PLOT_DIR}/h2_lmp_heatmap_STST_EXP.png")

### Price duration curves

In [ ]:
# get lmps of selected buses
lmp_stst = pd.DataFrame(index=stst.buses_t.marginal_price.index)
lmp_exp = pd.DataFrame(index=exp.buses_t.marginal_price.index)
c_buses_h2 = [ "H2"]

for c in c_buses_h2:
    lmp_stst[c]= stst.buses_t.marginal_price.loc[:, stst.buses.carrier == c].mean(axis=1)
    lmp_exp[c]= exp.buses_t.marginal_price.loc[:, exp.buses.carrier == c].mean(axis=1)

In [ ]:
bus_colors_stst = { "H2": 'turquoise'}
bus_colors_exp = { "AC": 'darkslategrey'}

fig, axs = plt.subplots(ncols=1, nrows=2, figsize=(8,8))

for i, ax in enumerate(axs):

    c = "H2"
    duration_curve = pd.DataFrame(np.sort(stst.buses_t.marginal_price.loc[: , stst.buses.carrier == c], axis=None)[::-1]).set_index(pd.Index(np.linspace(0, 1, num=181*2920)))
    ax.plot(duration_curve, label=f"{c} (stst)", color="cadetblue")
    duration_curve = pd.DataFrame(np.sort(exp.buses_t.marginal_price.loc[: , exp.buses.carrier == c], axis=None)[::-1]).set_index(pd.Index(np.linspace(0, 1, num=181*2920)))
    ax.plot(duration_curve, label=f"{c} (exp)", color="hotpink")
    ax.legend()
    ax.set_title(f"{c}", fontsize=16, **font1)

    if i == 0:
        ax.set_ylim([-50,400])
        lim_str = "ylim=400"
    elif i == 1:
        ax.set_xlim([0, 0.02])
        lim_str = "xlim=0.02"

    ax.set_ylabel("Lmp ($€/MWh_{h2}$)")
    ax.set_xlabel("Fraction of total time")
    ax.set_title(f"Price duration curves of hydrogen bus ({lim_str})", fontsize=16, **font1)

fig.tight_layout()
plt.show()

fig.savefig(f"{PLOT_DIR}/lmp_h2_price_dur_xlim=0.02_ylim=400.png")

In [ ]:
from scipy import stats

df_exp_lmp = pd.DataFrame(np.sort(exp.buses_t.marginal_price.loc[: , exp.buses.carrier == "AC"], axis=None)[::-1])
df_stst_lmp = pd.DataFrame(np.sort(stst.buses_t.marginal_price.loc[: , stst.buses.carrier == "AC"], axis=None)[::-1])

#df_stst_lmp[df_stst_lmp < 0].dropna()
#df_exp_lmp[df_exp_lmp < 0].dropna()
#df_stst_lmp.max()
#df_exp_lmp.max()

# STST: 338 (< 0); 2674 (>400); 1286 (>1000);311 (>1500); 193 (>2000); 9 (>2500); 0 (>3000)
# EXP : 497 (< 0); 3054 (>400); 986 (>1000); 674 (>1500); 3 (>2000); 0 (>2500); 0 (>3000)

In [ ]:
stst.buses_t.marginal_price.loc[: , stst.buses.carrier == "H2"].sum(axis=1).idxmin()
#exp.buses_t.marginal_price.loc[: , exp.buses.carrier == "H2"].sum(axis=1).idxmin()

In [ ]:
# date with the highest sum of lmps: "2013-01-24 15:00:00"
stst.buses_t.marginal_price.loc[: , stst.buses.carrier == "H2"].loc["2013-01-24 15:00:00"].sum()
# date with the lowest sum of lmps: '2013-06-23 06:00:00'
#stst.buses_t.marginal_price.loc[: , stst.buses.carrier == "H2"].loc['2013-06-23 06:00:00'].sum()

In [ ]:
df_stst_ons["H2_lmp_sum_max"] = stst.buses_t.marginal_price.loc[: , stst.buses.carrier == "AC"].loc["2013-01-24 15:00:00"]
df_stst_ons["H2_lmp_sum_min"] = stst.buses_t.marginal_price.loc[: , stst.buses.carrier == "AC"].loc['2013-06-23 06:00:00']

In [ ]:
carriers = ["H2_lmp_sum_min", "H2_lmp_sum_max"]
df = df_stst_ons

fig, axs = plt.subplots(ncols=2, nrows=1, subplot_kw={'projection': ccrs.EqualEarth()},figsize=(18, 8))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs):

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}",
                                   ax=ax,
                                   cmap=plt.get_cmap("magma_r"),
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"market values ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carriers[i]}", fontsize=16, **font1)

fig.tight_layout()

#plt.close()
plt.show()

In [ ]:
#TODO: investigate further into very high and very low prices

### Price bands

In [ ]:
# plot percentage of production per price band and percentage of revenue per price band
c_gen= c_h2_gen
c_con = c_h2_con
c_labels = c_con + c_gen
price_bands = [-np.inf, 50, 75, 100, 125, 150, np.inf]
price_bands_labels = ["< 50","[50 - 75)", "[75 - 100)", "[100 - 125)", "[125 - 150)", ">= 150"]
res_gen_con_stst = pd.DataFrame(index=[str(s) for s in range(len(price_bands)-1)])
res_rev_cost_stst = pd.DataFrame(index=[str(s) for s in range(len(price_bands)-1)])
res_gen_con_exp = pd.DataFrame(index=[str(s) for s in range(len(price_bands)-1)])
res_rev_cost_exp = pd.DataFrame(index=[str(s) for s in range(len(price_bands)-1)])
buses = ["bus0", "bus1", "bus2", "bus3", "bus4"]
ps = ["p0", "p1", "p2", "p3", "p4"]

In [ ]:
# calc

for n, res_gen_con, res_rev_cost in zip([stst, exp], [res_gen_con_stst, res_gen_con_exp], [res_rev_cost_stst, res_rev_cost_exp]):
    for c in c_labels:
        if c in n.generators.carrier.unique().tolist():
            c_bus = n.generators[n.generators.carrier == c].bus.map(n.buses.carrier).unique()[0]
            c_tag = c_tags[c_bus]
            gen_tag = "gen"
            gen = n.generators_t.p.loc[:, n.generators.carrier == c]
            gen.columns = gen.columns.map(n.generators.bus)

            lmp = n.buses_t.marginal_price.loc[:, gen.columns]
            rev = gen * lmp

            for i in range(len(price_bands)-1):
                lmp_cat = lmp.mask((lmp >= price_bands[i]) & (lmp < price_bands[i+1]), "hit")
                res_gen_con.loc[str(i), f"{c}_{gen_tag}_{c_tag}_{c_bus}"] = gen[lmp_cat == "hit"].sum().sum() / gen.sum().sum()
                res_rev_cost.loc[str(i), f"{c}_{gen_tag}_{c_tag}_{c_bus}"] = rev[lmp_cat == "hit"].sum().sum() / rev.sum().sum()

        elif c in n.links.carrier.unique().tolist():
            for i, bus in enumerate(buses):
                # check if bus exists
                if n.links[n.links.carrier == c][bus][0] != "":
                    c_bus = n.links[n.links.carrier == c][bus].map(n.buses.carrier).unique()[0]
                    c_tag = c_tags[c_bus]
                    # check if consumption or generation
                    gen = generation_links_bus(n, c, i)
                    gen_tag = "gen" if gen.sum() > 0 else "con"
                    if gen_tag == "gen":
                        gen = n.links_t[f"{ps[i]}"].loc[:, n.links.carrier == c] * -1
                    elif gen_tag == "con":
                        gen = n.links_t[f"{ps[i]}"].loc[:, n.links.carrier == c]

                    gen.columns = gen.columns.map(n.links[bus])

                    lmp = n.buses_t.marginal_price.loc[:, gen.columns]
                    rev = gen * lmp

                    for i in range(len(price_bands)-1):
                        lmp_cat = lmp.mask((lmp >= price_bands[i]) & (lmp < price_bands[i+1]), "hit")
                        res_gen_con.loc[str(i), f"{c}_{gen_tag}_{c_tag}_{c_bus}"] = gen[lmp_cat == "hit"].sum().sum() / gen.sum().sum()
                        res_rev_cost.loc[str(i), f"{c}_{gen_tag}_{c_tag}_{c_bus}"] = rev[lmp_cat == "hit"].sum().sum() / rev.sum().sum()


        elif c in n.storage_units.carrier.unique().tolist():
            c_bus = n.storage_units[n.storage_units.carrier == c].bus.map(n.buses.carrier).unique()[0]
            c_tag = c_tags[c_bus]
            for gen_tag in ["gen", "con"]:
                if gen_tag == "gen":
                    gen = n.storage_units_t.p_dispatch.loc[:, n.storage_units.carrier == c]
                elif gen_tag == "con":
                    gen = n.storage_units_t.p_store.loc[:, n.storage_units.carrier == c]

                gen.columns = gen.columns.map(n.storage_units.bus)

                lmp = n.buses_t.marginal_price.loc[:, gen.columns]
                rev = gen * lmp

                for i in range(len(price_bands)-1):
                    lmp_cat = lmp.mask((lmp >= price_bands[i]) & (lmp < price_bands[i+1]), "hit")
                    res_gen_con.loc[str(i), f"{c}_{gen_tag}_{c_tag}_{c_bus}"] = gen[lmp_cat == "hit"].sum().sum() / gen.sum().sum()
                    res_rev_cost.loc[str(i), f"{c}_{gen_tag}_{c_tag}_{c_bus}"] = rev[lmp_cat == "hit"].sum().sum() / rev.sum().sum()

        else:
            print(f"{c} not known!")

In [ ]:
round(res_gen_con_stst, 4)

In [ ]:
round(res_rev_cost_stst, 4)

In [ ]:
# extract right labels
con_labels = [ 'H2 liquefaction_con_h2_H2', 'Sabatier_con_h2_H2', 'Fischer-Tropsch_con_h2_H2']
gen_labels = ['H2 Electrolysis_gen_h2_H2', 'SMR CC_gen_h2_H2']

In [ ]:
# consumption and generation

model = "STST"

if model == "STST":
    res_gen_con = res_gen_con_stst
    res_rev_cost = res_rev_cost_stst
    gen_labels_index = gen_labels

elif model == "EXP":
    res_gen_con = res_gen_con_exp
    res_rev_cost = res_rev_cost_exp
    gen_labels_index = ['H2 Electrolysis_gen_h2_H2']

con_c = pd.Index(con_labels).str.split('_').str[0]
gen_c = pd.Index(gen_labels_index).str.split('_').str[0]

ticks = [carrier_renaming.get(n, n) for n in (con_c.to_list() + gen_c.to_list())]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(18, 8))

gen_con = res_gen_con[con_labels + gen_labels_index]
rev_cost = res_rev_cost[con_labels + gen_labels_index]
col = sns.color_palette("inferno_r", 6)

ax1 = gen_con.transpose().plot(kind='bar', stacked=True, color=col, ax=ax1, legend=False)
ax1.set_ylabel("Consumed or delivered hydrogen [%]")
ax1.set_xticks(ticks=range(len(ticks)), labels=ticks, rotation=0)
ax1.set_title(f"Consumption or Generation by hydrogen price band ({model})", fontsize=16, **font1, pad=10)
ax1.set_ylim([0,1])
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

ax2 = rev_cost.transpose().plot(kind='bar', stacked=True, color=col, ax=ax2)
ax2.set_ylabel("Cost or revenue [%]")
ax2.legend(title='Price band ($€/MWh_{h2}$)', labels=price_bands_labels , bbox_to_anchor=(1, 1))
ax2.set_xticks(ticks=range(len(ticks)), labels=ticks, rotation=0)
ax2.set_title(f"Cost or revenue by hydrogen price band ({model})", fontsize=16, **font1, pad=10)

ax2.axvline(x=len(c_con)-0.5,ymin=0,ymax=1,c="red",linewidth=2,linestyle="--", zorder=3, clip_on=False, label="seperator")
ax2.axvline(x=len(c_con)-0.5,ymin=1.27,ymax=2.27,c="red",linewidth=2,linestyle="--", zorder=3, clip_on=False, label="seperator")
ax2.set_ylim([0,1])
ax2.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

fig.tight_layout(pad=2)

plt.show()

fig.savefig(f"{PLOT_DIR}/h2_price_bands_all_{model}.png")

In [ ]:
round(res_gen_con_stst[gen_labels], 4)#.mean(axis=1)#.loc["3":].sum()#.loc["3":,:].sum()
round(res_rev_cost_stst[gen_labels], 4)#.mean(axis=1)

In [ ]:
round(res_gen_con_exp[gen_labels], 4)#.mean(axis=1)#.loc["3":].sum()#.loc["3":,:].sum()
round(res_rev_cost_exp[gen_labels], 4).mean(axis=1)

In [ ]:
round(res_gen_con_stst[con_labels], 4) - round(res_gen_con_exp[con_labels], 4)

In [ ]:
#round(res_gen_con_exp[gen_peak], 4).loc["3":,:].sum()#.loc["2":"3",:].sum().mean() #.mean(axis=1)  #.loc["3":].sum()#.loc["3":,:].sum()
#round(res_rev_cost_exp[gen_labels], 4).mean(axis=1)

In [ ]:
(round(res_gen_con_stst[gen_labels], 4) - round(res_gen_con_exp[gen_labels], 4))#.min()

## Temporal and spatio-temporal differences

#### Comparison between the models

In [ ]:
# hydrogen generation

In [ ]:
carriers = ["H2 Electrolysis", "SMR CC"]
models = ["STST", "EXP"]
colors = ["cadetblue", "hotpink"]

fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(12, 1*4))

for c, ax in zip(carriers, axs.reshape(-1)):

    for i, n in enumerate([stst,exp]):
        if c in n.generators.carrier.unique():
            c_bus = n.generators[n.generators.carrier == c].bus.map(n.buses.carrier).unique()[0]
            gen = n.generators_t.p.loc[: , n.generators.carrier == c]
            lmps = n.buses_t.marginal_price.loc[:, gen.columns.map(n.generators.bus)]
            df = pd.concat([pd.DataFrame(lmps.values.flatten()) , pd.DataFrame(gen.values.flatten())], axis=1)

        elif c in n.links.carrier.unique():
            c_bus = n.links[n.links.carrier == c]["bus1"].map(n.buses.carrier).unique()[0]
            gen = n.links_t.p1.loc[: , n.links.carrier == c]
            lmps = n.buses_t.marginal_price.loc[:, gen.columns.map(n.links.bus1)]
            df = pd.concat([pd.DataFrame(lmps.values.flatten()) , pd.DataFrame(gen.values.flatten())], axis=1)

        elif c in n.storage_units.carrier.unique():
            c_bus = n.storage_units[n.storage_units.carrier == c].bus.map(n.buses.carrier).unique()[0]
            gen = n.storage_units_t.p_dispatch.loc[: , n.storage_units.carrier == c]
            lmps = n.buses_t.marginal_price.loc[:, gen.columns.map(n.storage_units.bus)]
            df = pd.concat([pd.DataFrame(lmps.values.flatten()) , pd.DataFrame(gen.values.flatten())], axis=1)

        df.columns = ["lmp" , "gen"]
        df = df.sort_values(by="lmp")
        df["gen_cumsum"] = df["gen"].cumsum()
        df["gen_cumsum_norm"] = df["gen"].cumsum() / df["gen"].sum()
        ax.plot(df["gen_cumsum_norm"], df["lmp"], label=models[i], color=colors[i])
        ax.set_ylim([-50, 400])
        # add corridor which contains 75 % of the generation around the median
        ax.hlines(df["lmp"].loc[df["lmp"][df["gen_cumsum_norm"] > 0.125].index[0]], 0, 1, color=colors[i], ls="--", lw=1)
        ax.hlines(df["lmp"].loc[df["lmp"][df["gen_cumsum_norm"] > 0.875].index[0]], 0, 1,  color=colors[i], ls="--", lw =1)
        ax.set_ylabel("lmp [$€/MWh_{el}$")
        ax.set_xlabel("Fraction of total generation")
        ax.set_title(f"{c} (bus = {c_bus})", fontsize=16, **font1)
        ax.legend()

fig.tight_layout(pad=3)
plt.show()

In [ ]:
# hydrogen consumption

In [ ]:
carriers = c_h2_con
models = ["STST", "EXP"]
colors = ["cadetblue", "hotpink"]

fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(12, 2*4))

for c, ax in zip(carriers, axs.reshape(-1)):

    for i, n in enumerate([stst,exp]):

        if c in n.links.carrier.unique():
            c_bus = n.links[n.links.carrier == c]["bus0"].map(n.buses.carrier).unique()[0]
            con = n.links_t.p0.loc[: , n.links.carrier == c]
            lmps = n.buses_t.marginal_price.loc[:, con.columns.map(n.links.bus0)]
            df = pd.concat([pd.DataFrame(lmps.values.flatten()) , pd.DataFrame(con.values.flatten())], axis=1)

        elif c in n.storage_units.carrier.unique():
            c_bus = n.storage_units[n.storage_units.carrier == c].bus.map(n.buses.carrier).unique()[0]
            con = n.storage_units_t.p_store.loc[: , n.storage_units.carrier == c]
            lmps = n.buses_t.marginal_price.loc[:, con.columns.map(n.storage_units.bus)]
            df = pd.concat([pd.DataFrame(lmps.values.flatten()) , pd.DataFrame(con.values.flatten())], axis=1)

        df.columns = ["lmp" , "gen"]
        df = df.sort_values(by="lmp")
        df["gen_cumsum"] = df["gen"].cumsum()
        df["gen_cumsum_norm"] = df["gen"].cumsum() / df["gen"].sum()
        ax.plot(df["gen_cumsum_norm"], df["lmp"], label=models[i], color=colors[i])
        ax.set_ylim([-50, 400])
        # add corridor which contains 75 % of the generation around the median
        ax.hlines(df["lmp"].loc[df["lmp"][df["gen_cumsum_norm"] > 0.125].index[0]], 0, 1, color=colors[i], ls="--", lw=1)
        ax.hlines(df["lmp"].loc[df["lmp"][df["gen_cumsum_norm"] > 0.875].index[0]], 0, 1,  color=colors[i], ls="--", lw =1)
        ax.set_ylabel("lmp [$€/MWh_{el}")
        ax.set_xlabel("Fraction of total consumption")
        ax.set_title(f"{c} (bus = {c_bus})", fontsize=16, **font1)
        ax.legend()

fig.tight_layout(pad=3)
plt.show()

In [ ]:
# hydrogen generation adn consumption

In [ ]:
carriers = ["H2 Electrolysis"] + c_h2_con
models = ["STST", "EXP"]
colors = ["cadetblue", "hotpink"]

fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(12, 2*4))

for c, ax in zip(carriers, axs.reshape(-1)):

    for i, n in enumerate([stst,exp]):
        # generation
        if c == "H2 Electrolysis":

            if c in n.links.carrier.unique():
                c_bus = n.links[n.links.carrier == c]["bus1"].map(n.buses.carrier).unique()[0]
                gen = n.links_t.p1.loc[: , n.links.carrier == c]
                lmps = n.buses_t.marginal_price.loc[:, gen.columns.map(n.links.bus1)]
                df = pd.concat([pd.DataFrame(lmps.values.flatten()) , pd.DataFrame(gen.values.flatten())], axis=1)

            df.columns = ["lmp" , "gen"]
            df = df.sort_values(by="lmp")
            df["gen_cumsum"] = df["gen"].cumsum()
            df["gen_cumsum_norm"] = df["gen"].cumsum() / df["gen"].sum()
            ax.plot(df["gen_cumsum_norm"], df["lmp"], label=models[i], color=colors[i])
            ax.set_ylim([-50, 400])
            # add corridor which contains 75 % of the generation around the median
            ax.hlines(df["lmp"].loc[df["lmp"][df["gen_cumsum_norm"] > 0.125].index[0]], 0, 1, color=colors[i], ls="--", lw=1)
            ax.hlines(df["lmp"].loc[df["lmp"][df["gen_cumsum_norm"] > 0.875].index[0]], 0, 1,  color=colors[i], ls="--", lw =1)
            ax.set_ylabel("lmp [$€/MWh_{h2}$")
            ax.set_xlabel("Fraction of total generation")
            ax.set_title(f"{c} (bus = {c_bus})", fontsize=16, **font1)
            ax.legend()

        else:

            if c in n.links.carrier.unique():
                c_bus = n.links[n.links.carrier == c]["bus0"].map(n.buses.carrier).unique()[0]
                con = n.links_t.p0.loc[: , n.links.carrier == c]
                lmps = n.buses_t.marginal_price.loc[:, con.columns.map(n.links.bus0)]
                df = pd.concat([pd.DataFrame(lmps.values.flatten()) , pd.DataFrame(con.values.flatten())], axis=1)

            df.columns = ["lmp" , "gen"]
            df = df.sort_values(by="lmp")
            df["gen_cumsum"] = df["gen"].cumsum()
            df["gen_cumsum_norm"] = df["gen"].cumsum() / df["gen"].sum()
            ax.plot(df["gen_cumsum_norm"], df["lmp"], label=models[i], color=colors[i])
            ax.set_ylim([-50, 400])
            # add corridor which contains 75 % of the generation around the median
            ax.hlines(df["lmp"].loc[df["lmp"][df["gen_cumsum_norm"] > 0.125].index[0]], 0, 1, color=colors[i], ls="--", lw=1)
            ax.hlines(df["lmp"].loc[df["lmp"][df["gen_cumsum_norm"] > 0.875].index[0]], 0, 1,  color=colors[i], ls="--", lw =1)
            ax.set_ylabel("lmp [$€/MWh_{h2}")
            ax.set_xlabel("Fraction of total consumption")
            ax.set_title(f"{c} (bus = {c_bus})", fontsize=16, **font1)
            ax.legend()

fig.tight_layout(pad=3)
plt.show()

fig.savefig(f"{PLOT_DIR}/h2_con_gen_w_lmps_between_model.png")

#### Comparison within the models

In [ ]:
# generation

In [ ]:
carriers_stst = ["H2 Electrolysis", "SMR CC"]
carriers_exp = ["H2 Electrolysis"]

fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(14, 5))

for n, ax, model in zip([stst,exp], axs.reshape(-1), ["STST", "EXP"]):

    if model == "STST":
        carriers = carriers_stst

    elif model == "EXP":
        carriers = carriers_exp

    for c in carriers:
        if c in n.generators.carrier.unique():
            c_bus = n.generators[n.generators.carrier == c].bus.map(n.buses.carrier).unique()[0]
            gen = n.generators_t.p.loc[: , n.generators.carrier == c]
            lmps = n.buses_t.marginal_price.loc[:, gen.columns.map(n.generators.bus)]
            df = pd.concat([pd.DataFrame(lmps.values.flatten()) , pd.DataFrame(gen.values.flatten())], axis=1)

        elif c in n.links.carrier.unique():
            c_bus = n.links[n.links.carrier == c]["bus1"].map(n.buses.carrier).unique()[0]
            gen = n.links_t.p1.loc[: , n.links.carrier == c]
            lmps = n.buses_t.marginal_price.loc[:, gen.columns.map(n.links.bus1)]
            df = pd.concat([pd.DataFrame(lmps.values.flatten()) , pd.DataFrame(gen.values.flatten())], axis=1)

        elif c in n.storage_units.carrier.unique():
            c_bus = n.storage_units[n.storage_units.carrier == c].bus.map(n.buses.carrier).unique()[0]
            gen = n.storage_units_t.p_dispatch.loc[: , n.storage_units.carrier == c]
            lmps = n.buses_t.marginal_price.loc[:, gen.columns.map(n.storage_units.bus)]
            df = pd.concat([pd.DataFrame(lmps.values.flatten()) , pd.DataFrame(gen.values.flatten())], axis=1)

        df.columns = ["lmp" , "gen"]
        df = df.sort_values(by="lmp")
        df["gen_cumsum"] = df["gen"].cumsum()
        df["gen_cumsum_norm"] = df["gen"].cumsum() / df["gen"].sum()
        ax.plot(df["gen_cumsum_norm"], df["lmp"], color=carrier_colors[c], label= f"{c} ({c_bus})",
                marker=markers[carriers.index(c)], markevery=2920*40)
        ax.set_ylim([-50, 500])
        ax.set_ylabel("lmp [$€/MWh_{h2}$]")
        ax.set_xlabel("Fraction of total generation")
        ax.set_facecolor("whitesmoke")
        ax.set_title(f"Hydrogen generating technologies ({model})", fontsize=16, **font1)
        ax.legend()

fig.tight_layout(pad=3)
plt.show()

fig.savefig(f"{PLOT_DIR}/h2_gen_w_lmps_within_models.png")

In [ ]:
# hydrogen consuming technologies

In [ ]:
carriers = c_h2_con

fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(14, 5))

for n, ax, model in zip([stst,exp], axs.reshape(-1), ["STST", "EXP"]):

    for c in carriers:
        if c in n.links.carrier.unique():
            c_bus = n.links[n.links.carrier == c]["bus0"].map(n.buses.carrier).unique()[0]
            con = n.links_t.p0.loc[: , n.links.carrier == c]
            lmps = n.buses_t.marginal_price.loc[:, con.columns.map(n.links.bus0)]
            df = pd.concat([pd.DataFrame(lmps.values.flatten()) , pd.DataFrame(con.values.flatten())], axis=1)

        elif c in n.storage_units.carrier.unique():
            c_bus = n.storage_units[n.storage_units.carrier == c].bus.map(n.buses.carrier).unique()[0]
            con = n.storage_units_t.p_store.loc[: , n.storage_units.carrier == c]
            lmps = n.buses_t.marginal_price.loc[:, con.columns.map(n.storage_units.bus)]
            df = pd.concat([pd.DataFrame(lmps.values.flatten()) , pd.DataFrame(con.values.flatten())], axis=1)

        df.columns = ["lmp" , "gen"]
        df = df.sort_values(by="lmp")
        df["gen_cumsum"] = df["gen"].cumsum()
        df["gen_cumsum_norm"] = df["gen"].cumsum() / df["gen"].sum()
        ax.plot(df["gen_cumsum_norm"], df["lmp"], color=carrier_colors[c], label= f"{c} ({c_bus})",
                marker=markers[carriers.index(c)], markevery=2920*40)
        ax.set_ylim([-50, 300])
        ax.set_ylabel("lmp [$€/MWh_{h2}$]")
        ax.set_xlabel("Fraction of total consumption")
        ax.set_facecolor("whitesmoke")
        ax.set_title(f"Hydrogen consuming technologies ({model})", fontsize=16, **font1)
        ax.legend()

fig.tight_layout(pad=3)
plt.show()

fig.savefig(f"{PLOT_DIR}/h2_con_w_lmps_within_models.png")